## 1. Imports

In [5]:
import requests
import pandas as pd
import re

from datetime import datetime
from bs4 import BeautifulSoup as bs

import sqlite3
from sqlalchemy import create_engine

## 2. Functions

### 2.1 Scraping the categories and urls desired

In [2]:
def catalogue_urls(url):
    # This function intend to get the name and url of the book categories and put them in a dataframe 
    # that will be used afterwards
    
    # header to look real
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    
    # Access the root page and store it
    page = requests.get( url, headers=headers)
    
    # BeautifulSoup Object
    soup = bs( page.text, 'html.parser')
    
    #This is where I found the list including all the categories in the sidebar    
    catalogue_list = soup.find( 'ul', class_='nav nav-list').find_all('a')
    
    # Scraping the urls from all the categories
    catalogue_urls = [p.get('href') for p in catalogue_list]
    
    # Scraping the category names
    catalogue_list = [p.get_text().split() for p in catalogue_list]
    catalogue_list = [' '.join(strings) for strings in catalogue_list]
    
    # Creating a dataframe using the category names and the urls and filtering to get only the ones I need.
    catalogue_df = pd.DataFrame({'Catalogue':catalogue_list, 'urls': catalogue_urls})
    catalogue_true = ['Classics', 'Science Fiction', 'Humor', 'Business']
    catalogue = catalogue_df[catalogue_df['Catalogue'].isin(catalogue_true)]
    
    return catalogue

### 2.2 Book scraping

In [9]:
def scrapping(catalogue):
    # This function use the book categories and urls from the previous function to scrap the books informations

    # Creating an empty dataframe
    books_df_final = pd.DataFrame()
    
    # Root url
    root_url = 'https://books.toscrape.com/'
    
    # header to look real
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        
    for index, row in catalogue.iterrows():
        
        #extracting infos from the catalogue
        cat = row['Catalogue']
        end_url = row['urls']
        
        # uniting the two parts of the url 
        url = root_url + end_url
        
        # Access and store the page
        page = requests.get( url, headers=headers)

        # BeautifulSoup Object
        soup = bs( page.text, 'html.parser')

        #This is where I found the list of all books from the category
        books = soup.find( 'ol', class_='row')

        # Creating an empty dataframe
        books_df = pd.DataFrame()

        # Book title
        books_df['book_title'] = [p.get_text( 'title' ) for p in books.find_all('h3')]
        
        # Book price
        books_df['book_price'] = [p.get_text() for p in books.find_all( 'p', class_='price_color')]
        books_df['book_price'][:] = [item.replace("Â£", "") for item in books_df['book_price']]
        books_df['book_price'] = books_df['book_price'].astype( float )

        # Book rate
        rating_list = (soup.find('ol', class_='row')).find_all('p', class_='star-rating')
        number_list = ['One', 'Two', 'Three', 'Four', 'Five']
        number_dict = {'One':1, 'Two':2, 'Three':3, 'Four':4, 'Five':5}

        regex = re.compile('|'.join(re.escape(x) for x in number_list))
        books_df['book_rate'] = re.findall(regex,str(rating_list))

        # changing the string type rate to number using dict comprehension
        books_df['book_rate'] = books_df['book_rate'].replace(number_dict) 

        # Book Availability
        books_df['book_availability'] = [p.get_text().strip() for p in books.find_all( 'p', class_='instock availability')]

        # Category name
        books_df['book_category'] = cat
        #books_df['book_category'] = 'CHANGE HERE'

        # datetime of scrap
        scrap_time = datetime.now().strftime( '%Y-%m-%d %H:%M:%S' )
        books_df['scrap_time'] = scrap_time
        books_df['scrap_time'] = pd.to_datetime( books_df['scrap_time'], format='%Y-%m-%d %H:%M:%S')
        
        # Appending all the dataframes into one
        books_df_final = books_df_final.append(books_df)
        
    return books_df_final

## 3. Scrap

In [10]:
# defining the root url
url = 'https://books.toscrape.com/'

# running the catalogue_urls function to get the urls of the different categories
catalogue = catalogue_urls(url)

# running the webscraping function
books_df = scrapping(catalogue)
books_df

,book_title,book_price,book_rate,book_availability,book_category,scrap_time
0,The Secret Garden,15.08,4,In stock,Classics,2022-03-14 19:17:43
1,The Metamorphosis,28.58,1,In stock,Classics,2022-03-14 19:17:43
2,The Pilgrim's Progress,50.26,2,In stock,Classics,2022-03-14 19:17:43
3,The Hound of the ...,14.82,2,In stock,Classics,2022-03-14 19:17:43
4,Little Women (Little Women ...,28.07,4,In stock,Classics,2022-03-14 19:17:43
5,Gone with the Wind,32.49,3,In stock,Classics,2022-03-14 19:17:43
6,Candide,58.63,3,In stock,Classics,2022-03-14 19:17:43
7,Animal Farm,57.22,3,In stock,Classics,2022-03-14 19:17:43
8,Wuthering Heights,17.73,3,In stock,Classics,2022-03-14 19:17:43
9,The Picture of Dorian ...,29.70,2,In stock,Classics,2022-03-14 19:17:43


In [44]:
# turning it into a csv file
csv_file = books_df.to_csv('TeaOClock-dataset.csv')

## 4. Store in a Database


In [11]:
# check datatupe
books_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 0 to 11
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   book_title         57 non-null     object        
 1   book_price         57 non-null     float64       
 2   book_rate          57 non-null     int64         
 3   book_availability  57 non-null     object        
 4   book_category      57 non-null     object        
 5   scrap_time         57 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 3.1+ KB


In [12]:
# query to create the table and columns
query_book_schema = """
    CREATE TABLE book_schema (
        book_title            TEXT, 
        book_price            REAL, 
        book_rate             INTEGER, 
        book_availability     TEXT,
        book_category         TEXT, 
        scrap_time            TEXT
    )
"""

In [13]:
# connect to database
conn = sqlite3.connect( 'books_db.sqlite ')
cursor = conn.execute( query_book_schema )
conn.commit()
cursor.close()

In [15]:
# create connector to make some queries
conn = create_engine( 'sqlite:///books_db.sqlite', echo=False )

In [16]:
# show the empty table
query = """
    SELECT * FROM book_schema
"""
df_bd = pd.read_sql_query( query, conn )
df_bd

,book_title,book_price,book_rate,book_availability,book_category,scrap_time


In [19]:
# insert data into the table
books_df.to_sql( 'book_schema', con=conn, if_exists='append', index=False )

In [20]:
# show the empty table
query = """
    SELECT * FROM book_schema
"""
df_bd = pd.read_sql_query( query, conn )
df_bd

,book_title,book_price,book_rate,book_availability,book_category,scrap_time
0,The Secret Garden,15.08,4,In stock,Classics,2022-03-14 19:17:43.000000
1,The Metamorphosis,28.58,1,In stock,Classics,2022-03-14 19:17:43.000000
2,The Pilgrim's Progress,50.26,2,In stock,Classics,2022-03-14 19:17:43.000000
3,The Hound of the ...,14.82,2,In stock,Classics,2022-03-14 19:17:43.000000
4,Little Women (Little Women ...,28.07,4,In stock,Classics,2022-03-14 19:17:43.000000
5,Gone with the Wind,32.49,3,In stock,Classics,2022-03-14 19:17:43.000000
6,Candide,58.63,3,In stock,Classics,2022-03-14 19:17:43.000000
7,Animal Farm,57.22,3,In stock,Classics,2022-03-14 19:17:43.000000
8,Wuthering Heights,17.73,3,In stock,Classics,2022-03-14 19:17:43.000000
9,The Picture of Dorian ...,29.70,2,In stock,Classics,2022-03-14 19:17:43.000000
